In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.common.exceptions import TimeoutException
from time import sleep
import os

chrome_driver = '/Users/nicholasdevito/Dropbox/Python projects/ANZCTR/chromedriver'
download_directory = '/Users/nicholasdevito/Desktop/Test ANZTR Test' #changed as needed for full program

In [ ]:
#low count ANZCTR test URL
test_anzctr = 'http://www.anzctr.org.au/TrialSearch.aspx?searchTxt=&conditionCategory=Ear&conditionCode=&interventionCodeOperator=OR&interventionCode=&ageGroup=&healthyVolunteers=&gender=All&allocationToIntervention=Randomised&dateOfRegistrationFrom=&dateOfRegistrationTo=&trialStartDateFrom=&trialStartDateTo=&recruitmentCountryOperator=OR&countryOfRecruitment=&primarySponsorType=&fundingSource=&healthCondition=&interventionDescription=&phase=&recruitmentStatus=&registry=ANZCTR&ethicsReview=&studyType=&isBasic=False&postcode=&distance='

#actual URL to use for full download
all_anzctr = 'http://www.anzctr.org.au/TrialSearch.aspx?searchTxt=&conditionCategory=&conditionCode=&interventionCodeOperator=OR&interventionCode=&ageGroup=&healthyVolunteers=&gender=&allocationToIntervention=&dateOfRegistrationFrom=&dateOfRegistrationTo=&trialStartDateFrom=&trialStartDateTo=&recruitmentCountryOperator=OR&countryOfRecruitment=&primarySponsorType=&fundingSource=&healthCondition=&interventionDescription=&phase=&recruitmentStatus=&registry=ANZCTR&ethicsReview=&studyType=&isBasic=False&postcode=&distance='

In [ ]:
#this gets the zip file from the ANZCTR

chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : download_directory}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=chrome_driver, options=chromeOptions)
driver.get(test_anzctr)
sleep(5)
driver.execute_script("document.getElementById('ctl00_body_btnDownload').click()")

#makes sure the file is finished downloading before quitting the driver
dl_check = 0
while dl_check == 0:
    count = 0
    for file in os.listdir(download_directory):
        if file.endswith(".crdownload"):
            count = count+1
    if count == 0:
        dl_check = 1
        driver.quit()
    else:
        dl_check = 0
        sleep(3)

In [ ]:
#gets the name of the downloaded zip file

zip_file = ''
for file in os.listdir(download_directory):
    if file.endswith('.zip'):
        zip_file = file

print(zip_file)


In [ ]:
#unzips it and then gets rid of it

import zipfile
zip_ref = zipfile.ZipFile(download_directory + '/' + zip_file, 'r')
zip_ref.extractall(download_directory)
zip_ref.close()

#cleans up xml file after extraction
for file in os.listdir(download_directory):
    if file.endswith('.zip'):
        os.remove(download_directory + '/' + zip_file)
#can also clean up .xsl file if wanted but probably not necessary

In [ ]:
import xmltodict
import json
import os
import pandas as pd

In [ ]:
anzctr_trials_list = []
directory = '/Users/nicholasdevito/Desktop/Test ANZTR Test' #make this the name or path of whatever the directory containing your files is from above

In [ ]:
for file in os.listdir(download_directory):
    if file.endswith('.xml'):
        with open(download_directory + '/' + file) as test:
            doc = xmltodict.parse(test.read())
            anzctr_trials_list.append(str(json.dumps(doc)))

In [ ]:
print(anzctr_trials_list[0])

In [ ]:
#this creates a csv with no headers and a single column with all trials in json in their own row.

import csv
with open('test.csv','w', newline = '')as test_csv:
    writer=csv.writer(test_csv)
    for val in anzctr_trials_list:
        writer.writerow([val])

In [ ]:
#next thing to think about: working with this list to extract info rather than using messy BQ and SQL